In [1]:
import os
import tarfile
import boto3
from urllib.parse import urlparse

import tensorflow as tf
import sagemaker
from sagemaker.tensorflow import TensorFlowModel

# =========================
# Config (YOUR BMI MODEL)
# =========================
H5_TAR_S3_URI = "s3://ai-bmi-predictor/trained-models/efficientnet-models/eff-ann-v8-training-2025-12-18-11-02-51-500/output/model.tar.gz"
ENDPOINT_NAME = "BMI-predcitor-v8"  # keeping your requested name
INSTANCE_TYPE = "ml.g4dn.xlarge"       # change to ml.g4dn.xlarge if you really need GPU
FRAMEWORK_VERSION = "2.11.0"

# Where to upload the converted SavedModel tar.gz
SAVEDMODEL_TAR_S3_URI = "s3://ai-bmi-predictor/trained-models/efficientnet-models/eff-ann-v8-training-2025-12-18-11-02-51-500/output/savedmodel/model.tar.gz"

# =========================
# Role inference (same pattern you had)
# =========================
try:
    ROLE  # type: ignore
except NameError:
    try:
        from sagemaker import get_execution_role
        ROLE = get_execution_role()
    except Exception:
        ROLE = None

if not ROLE:
    raise ValueError("ROLE is None. Set ROLE to your SageMaker execution role ARN if running outside SageMaker.")

# =========================
# Helpers
# =========================
def parse_s3_uri(uri: str):
    p = urlparse(uri)
    if p.scheme != "s3" or not p.netloc or not p.path:
        raise ValueError(f"Invalid S3 URI: {uri}")
    return p.netloc, p.path.lstrip("/")

def safe_extract(tar: tarfile.TarFile, path: str):
    abs_path = os.path.abspath(path)
    for member in tar.getmembers():
        member_path = os.path.abspath(os.path.join(path, member.name))
        if not member_path.startswith(abs_path + os.sep) and member_path != abs_path:
            raise Exception(f"Blocked path traversal attempt: {member.name}")
    tar.extractall(path)

def upload_file_to_s3(local_path: str, s3_uri: str):
    bucket, key = parse_s3_uri(s3_uri)
    boto3.client("s3").upload_file(local_path, bucket, key)
    return f"s3://{bucket}/{key}"

# =========================
# 1) Download & extract .h5 from your tar.gz
# =========================
workdir = "bmi_model_work"
os.makedirs(workdir, exist_ok=True)

local_in_tar = os.path.join(workdir, "model.tar.gz")
extract_dir = os.path.join(workdir, "extracted")
os.makedirs(extract_dir, exist_ok=True)

in_bucket, in_key = parse_s3_uri(H5_TAR_S3_URI)
boto3.client("s3").download_file(in_bucket, in_key, local_in_tar)
print(f"✅ Downloaded: {H5_TAR_S3_URI}")

with tarfile.open(local_in_tar, "r:gz") as tar:
    safe_extract(tar, extract_dir)

# Find the .h5 file
h5_path = None
for root, _, files in os.walk(extract_dir):
    for f in files:
        if f.lower().endswith((".h5", ".hdf5")):
            h5_path = os.path.join(root, f)
            break
    if h5_path:
        break

if not h5_path:
    raise ValueError("❌ No .h5 found inside the downloaded archive.")

print(f"✅ Found H5: {h5_path}")

# =========================
# 2) Convert H5 -> SavedModel (TF Serving layout: model/1/)
# =========================
model = tf.keras.models.load_model(h5_path, compile=False)
print("✅ Loaded Keras model")

serving_root = os.path.join(workdir, "model")   # will become the tar root
version_dir = os.path.join(serving_root, "1")   # TF Serving convention
os.makedirs(version_dir, exist_ok=True)

tf.saved_model.save(model, version_dir)
print(f"✅ Exported SavedModel to: {version_dir}")

# =========================
# 3) Re-pack as model.tar.gz (must contain 'model/1/...' inside)
# =========================
local_out_tar = os.path.join(workdir, "savedmodel.tar.gz")
with tarfile.open(local_out_tar, "w:gz") as tar:
    tar.add(serving_root, arcname="model")
print(f"✅ Repacked SavedModel tar: {local_out_tar}")

# =========================
# 4) Upload converted artifact to S3 (this is what SageMaker will deploy)
# =========================
uploaded_s3 = upload_file_to_s3(local_out_tar, SAVEDMODEL_TAR_S3_URI)
print(f"✅ Uploaded SavedModel artifact to: {uploaded_s3}")

# =========================
# 5) Deploy as SageMaker endpoint
# =========================
session = sagemaker.Session()
region = boto3.Session().region_name

tf_model = TensorFlowModel(
    model_data=uploaded_s3,
    role=ROLE,
    framework_version=FRAMEWORK_VERSION,
    sagemaker_session=session,
)

predictor = tf_model.deploy(
    initial_instance_count=1,
    instance_type=INSTANCE_TYPE,
    endpoint_name=ENDPOINT_NAME,
)

print(f"\n✅ Model successfully deployed to endpoint: {ENDPOINT_NAME}")
print(f"   • Region: {region}")
print(f"   • Artifact: {uploaded_s3}")


2025-12-18 11:43:05.635608: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-18 11:43:05.651091: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-18 11:43:05.675391: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-18 11:43:05.675423: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-18 11:43:05.690474: I tensorflow/core/platform/cpu_feature_gua

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
✅ Downloaded: s3://ai-bmi-predictor/trained-models/efficientnet-models/eff-ann-v8-training-2025-12-18-11-02-51-500/output/model.tar.gz
✅ Found H5: bmi_model_work/extracted/eff_ann_version8.h5


2025-12-18 11:43:11.360772: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-18 11:43:20.798560: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-18 11:43:20.801747: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

✅ Loaded Keras model
INFO:tensorflow:Assets written to: bmi_model_work/model/1/assets


INFO:tensorflow:Assets written to: bmi_model_work/model/1/assets


✅ Exported SavedModel to: bmi_model_work/model/1
✅ Repacked SavedModel tar: bmi_model_work/savedmodel.tar.gz
✅ Uploaded SavedModel artifact to: s3://ai-bmi-predictor/trained-models/efficientnet-models/eff-ann-v8-training-2025-12-18-11-02-51-500/output/savedmodel/model.tar.gz
---------!
✅ Model successfully deployed to endpoint: BMI-predcitor-v8
   • Region: eu-north-1
   • Artifact: s3://ai-bmi-predictor/trained-models/efficientnet-models/eff-ann-v8-training-2025-12-18-11-02-51-500/output/savedmodel/model.tar.gz
